In [1]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
from tqdm import tqdm
tqdm.pandas()
from surprise import Reader, Dataset, SVD
from sklearn.model_selection import GridSearchCV, cross_validate
svd = SVD()

c:\Users\maddy\anaconda3\envs\goodreads\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
read = pd.read_feather('Feather/read.feather')
books = pd.read_feather('Feather/books.feather')    
reviews = pd.read_feather('Feather/reviews.feather')
interactions = pd.read_feather('Feather/interactions.feather')

In [3]:
interactions = interactions[interactions['is_read']== True]

In [4]:
reviews = reviews.dropna(subset=['review_text', 'rating'])
user_review_counts = reviews.groupby('user_id').size()

users_with_more_than_3_reviews = user_review_counts[user_review_counts > 3].index

valid_reviews = reviews[reviews['user_id'].isin(users_with_more_than_3_reviews)]
valid_reviews = valid_reviews.head(10000)

In [5]:
len(reviews['review_text'])
reviews = reviews.dropna(subset=['review_text', 'rating'])
user_review_counts = reviews.groupby('user_id').size()

users_with_more_than_3_reviews = user_review_counts[user_review_counts > 3].index

valid_reviews = reviews[reviews['user_id'].isin(users_with_more_than_3_reviews)]
valid_reviews = valid_reviews.head(10000)


In [ ]:
import pandas as pd
from surprise import Reader, Dataset, SVD
from surprise.model_selection import GridSearchCV, cross_validate, train_test_split

# Prepare surprise dataset
reader = Reader()
data = Dataset.load_from_df(interactions[['user_id', 'book_id', 'rating']], reader)

# Split the data into training and test sets
trainset, testset = train_test_split(data, test_size=0.2)

# Define the parameter grid
param_grid = {
    'n_epochs': [10, 20],
    'lr_all': [0.001, 0.002],
    'reg_all': [0.01, 0.02],
    'n_factors': [20, 50,]  # Number of latent factors
}

# Initialize GridSearchCV
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=5)
gs.fit(data)

# Train SVD with the best hyperparameters
best_svd = gs.best_estimator['rmse']
cross_validate(best_svd, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

# Train on the full dataset
trainset = data.build_full_trainset()
best_svd.fit(trainset)


In [ ]:
# Merge genres into reviews
reviews_with_genres = valid_reviews.merge(books[['book_id', 'genres']], on='book_id', how='left')

# Combine review text and genres
reviews_with_genres['combined_text'] = reviews_with_genres.apply(lambda row: f"{row['review_text']} {row['genres']}", axis=1)

# Calculate combined embeddings for reviews
model = SentenceTransformer('all-MiniLM-L6-v2')
reviews_with_genres['combined_embeddings'] = reviews_with_genres['combined_text'].progress_apply(lambda x: model.encode(x))

# Combine combined_embeddings by user
user_embeddings = reviews_with_genres.groupby('user_id')['combined_embeddings'].apply(lambda x: np.mean(np.vstack(x), axis=0)).reset_index()

# Normalize ratings in training data
interactions['rating_normalized'] = (interactions['rating'] - interactions['rating'].min()) / (interactions['rating'].max() - interactions['rating'].min())

# Merge user embeddings with normalized ratings
user_ratings = interactions.groupby('user_id')['rating_normalized'].mean().reset_index()
combined_features = pd.merge(user_embeddings, user_ratings, on='user_id')


In [ ]:
combined_features.isnull().sum().sum()

In [ ]:
# Calculate User-User Similarity
user_features = np.vstack(combined_features['combined_embeddings'])
user_similarity = cosine_similarity(user_features, user_features)

In [ ]:

def recommend_books(user_id, user_similarity, interactions_df, books_df, combined_features, num_recommendations=5):
    # Check if user_id is in the combined_features dataframe
    if user_id not in combined_features['user_id'].values:
        return pd.DataFrame(columns=['book_id', 'title', 'authors', 'genres'])

    # Identify similar users
    user_index = combined_features[combined_features['user_id'] == user_id].index[0]
    similar_user_ids = combined_features['user_id'][user_similarity[user_index].argsort()[-num_recommendations-1:-1][::-1]].values

    # Get books read by similar users
    similar_users_books = interactions_df[interactions_df['user_id'].isin(similar_user_ids)]['book_id'].unique()

    # Filter out books already read by the user
    user_books = interactions_df[interactions_df['user_id'] == user_id]['book_id'].unique()
    recommended_books = [book_id for book_id in similar_users_books if book_id not in user_books]

    # Get details of the recommended books
    recommended_books_df = books_df[books_df['book_id'].isin(recommended_books)].head(num_recommendations)

    return recommended_books_df[['book_id', 'title']]


In [ ]:
recommend_books(4, user_similarity, interactions, books, combined_features)

In [ ]:
interactions[interactions['user_id']==4]